In [ ]:
!pip install ultralytics



from ultralytics import YOLO

# Load YOLOv8n (nano) model
model = YOLO('yolov8n.pt')  

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import os

def is_segmentation_line(line):
    parts = line.strip().split()
    return len(parts) > 5  # YOLO segmentation lines have more than 5 elements

labels_dir = "/kaggle/input/tumor-detection/train/labels"  # ← Change this to your label path

invalid_files = []

for file_name in os.listdir(labels_dir):
    if not file_name.endswith(".txt"):
        continue

    path = os.path.join(labels_dir, file_name)
    
    with open(path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]

    boxes = len(lines)
    segments = sum([1 for line in lines if is_segmentation_line(line)])

    if boxes != segments:
        invalid_files.append(file_name)

# ✅ Print mismatched label files
if invalid_files:
    print("🟥 Inconsistent label files (box count ≠ segment count):")
    for file in invalid_files:
        print(file)
    print(f"\nTotal invalid files: {len(invalid_files)}")
else:
    print("✅ All label files have matching box and segment counts.")


# Remove these images 

import shutil
import os

src = "/kaggle/input/tumor-detection"
dst = "/kaggle/working/tumor-detection-copy"

if not os.path.exists(dst):
    shutil.copytree(src, dst)

print("✅ Dataset copied to working directory.")


labels_dir = "/kaggle/working/tumor-detection-copy/train/labels"


import os

def is_segmentation_line(line):
    parts = line.strip().split()
    return len(parts) > 5  # YOLO segmentation has polygon points

labels_dir = "/kaggle/working/tumor-detection-copy/train/labels"  # or "val"
invalid_files = []

for file_name in os.listdir(labels_dir):
    if not file_name.endswith(".txt"):
        continue

    path = os.path.join(labels_dir, file_name)
    
    with open(path, 'r') as f:
        lines = f.readlines()

    boxes = 0
    segments = 0

    for line in lines:
        if line.strip() == "":
            continue
        if is_segmentation_line(line):
            boxes += 1
            segments += 1
        else:
            boxes += 1  # no segment found — box only

    if boxes != segments:
        invalid_files.append(file_name)
        os.remove(path)  # ❌ Remove invalid label file
        print(f"Removed mixed label file: {file_name}")

print("\n✅ Cleanup complete.")
print(f"Total removed files: {len(invalid_files)}")


# Train the model
model.train(
    data='/kaggle/input/tumor-detection/data.yaml',         # Path to your dataset YAML
    epochs=50,                # Number of epochs
    imgsz=640,                # Image size (can be 640 or other sizes like 512, 416)
    batch=16,                 # Batch size
    name='tumor_yolo_train', # Experiment name
    project='tumor_project', # Folder where runs will be saved
    device=0,                 # GPU (use 'cpu' if no GPU)
    workers=4,                # Data loading threads

    # Augmentations
    flipud=0.5,               # Vertical flip probability
    fliplr=0.5,               # Horizontal flip probability
    hsv_h=0.015,              # Hue variation
    hsv_s=0.7,                # Saturation variation
    hsv_v=0.4,                # Value (brightness) variation
    scale=0.5,                # Image scaling
    shear=0.0,                # Shear (disabled here, but can use 0.2)
    perspective=0.0,          # Perspective distortion
    degrees=0.0,              # Rotation angle

    # Optimization
    lr0=0.01,                 # Initial learning rate
    lrf=0.01,                 # Final learning rate (lr0 * lrf)
    momentum=0.937,           # Momentum
    weight_decay=0.0005,      # Weight decay
    optimizer='SGD',          # Optimizer: SGD | Adam | AdamW

    # Saving/Logging
    save=True,
    save_period=5,            # Save model every N epochs
    exist_ok=True             # Overwrite existing folder if it exists
)

# Run inference

model = YOLO('tumor_project/tumor_yolo_train/weights/best.pt')


results = model('/kaggle/input/tumor-detection/test/images/meningioma_1038_jpg.rf.1bbe682f1f254698336b40c2fcfb98f6.jpg', save=True, conf=0.25)


# show image 

results[0].show()

# save best weights 

model.save('/kaggle/working/best.pt')

# Get tensors 

for result in results:
    boxes=result.boxes
    print(boxes)


from ultralytics import SAM

# APPLY SAM MODEL FOR SEGMENTATION 

sam_model=SAM('sam2_b.pt')
for resutl in results:
    cls_ids=result.boxes.cls.int().tolist()
    if len(cls_ids):
        boxes=result.boxes.xyxy
        sam_results=sam_model(result.orig_img,bboxes=boxes,verbose=False,save=True,device=0)

SEE Prediction



from PIL import Image
import matplotlib.pyplot as plt

img_path = '/kaggle/working/runs/segment/predict/image0.jpg'  # Replace with your image path
image = Image.open(img_path)

plt.imshow(image)
plt.axis('off')
plt.show()
